In [1]:
import matplotlib.pyplot as plt
import matplotlib
import serial
import time
import sys 

In [3]:
class ZabberVoiceCoil:
    """Class for controlling the Zabber X-DMQ12P voice coil stage
    
    Attributes
    ----------
    
    Examples
    --------
    >>> stage = ZabberVoiceCoil('COM4')
    >>> stage.move_abs(1000)
    >>> stage.close()
    
    """
    def __init__(self,com = 'COM3'):
        """Initiates the connection to the stage
        
        
        """
        try:
            self.__ser = None
            self.__ser = serial.Serial(com, 115200 , timeout=1, parity=serial.PARITY_NONE)
            # check device ID
            self.__ser.write(('/get deviceid\r\n').encode('utf-8'))
            qry = self.__ser.readline()
            if qry.find(b'50419') == -1:
                print('Error reading device ID.')
                raise Exception()
            else:
                print('ID OK:' + str(qry)[2:-5])
       
        except:
            if self.__ser is not None:
                self.close()
            print('Something went wrong:')
            raise Exception('Error openning the communication')
            
            
           
    def home(self):
        """From any position to 0 
        """
        
        self.__ser.write(('/home \r\n').encode('utf-8'))
        qry = self.__ser.readline()
        print(qry)
      
    def move_abs(self,pos):
        """range 0(minimum position) .. 30000(mid position).. 60000(max position)
        
        """
        self.__ser.write(('/move abs {:d}  \r\n'.format(pos)).encode('utf-8'))
        qry = self.__ser.readline()
        print(qry)
        
    def set_velocity(self,vel):
        """range +ve or -ve (10..100..1000..10000)
           also depends on current position of stage
        """
        self.__ser.write(('/move vel {:d}  \r\n'.format(vel)).encode('utf-8'))
        qry = self.__ser.readline()
        print(qry)
        
    def stop(self):
        self.__ser.write(('/stop \r\n').encode('utf-8'))
        qry = self.__ser.readline()
        print(qry)
                
    def move_rel(self, dist):
        """-60000 to +60000(depends on current position of stage)
        """
        self.__ser.write(('/move abs {:d}  \r\n'.format(dist)).encode('utf-8'))
        qry = self.__ser.readline()
        print(qry)
        
    
    def sin(self,amp,period,center=30000):
        """ 1. Move to center position of stage
            2. Amp(amplitude):- +- 1000 micro meter or 10000 micro meter
            3. Period :- 100 ms or 1000 ms
        """
        self.move_abs(center)
    
        wait = True;
        while wait:
            self.__ser.write(('/get deviceid\r\n').encode('utf-8'))
            qry = self.__ser.readline()
            if qry.find(b'IDLE') != -1:
                wait = False
                time.sleep(0.5)
                
        self.__ser.write(('/move sin {:d} {} \r\n'.format(amp,period)).encode('utf-8'))
        qry = self.__ser.readline()
        print(qry)    
                
    def sin_stop(self):
        self.__ser.write(('/move sin stop \r\n').encode('utf-8'))
        qry = self.__ser.readline()
        print(qry)
    
    def close(self):
        if self.__ser.is_open:
            self.__ser.close()
            print('Communication closed...')
        else:
            print('Communication was not open...')

In [8]:
s = ZabberVoiceCoil('COM3')

ID OK:@01 0 OK IDLE NI 50419


In [30]:
s.home()                # From any position to 0  

b'@01 0 OK BUSY -- 0\r\n'


In [29]:
s.move_abs(38900)    # range 0(minimum position), dis_mov is 0.6 cm at 30000(mid position),1.2 cm 60000(max position)

b'@01 0 OK BUSY -- 0\r\n'


In [33]:
s.move_rel(30000) # range 0 to +60000(depends on current position of stage)

b'@01 0 OK BUSY -- 0\r\n'


In [14]:
s.set_velocity(10000)    # range +ve or -ve (10..100..1000..10000)(depends on current position of stage)

b'@01 0 OK BUSY -- 0\r\n'


In [5]:
s.stop()

b'@01 0 OK BUSY NI 0\r\n'


In [9]:
s.sin(1000,100)         # 1st move mid pos... amp(1000(1 mm) or 10000(10 mm)) period(100 ms or 1000 ms)

b'@01 0 OK BUSY -- 0\r\n'
b'@01 0 OK BUSY -- 0\r\n'


In [10]:
s.sin_stop()

b'@01 0 OK BUSY NI 0\r\n'


In [11]:
s.close()

Communication closed...


In [13]:
ser = serial.Serial('COM3', 115200 , timeout=1, parity=serial.PARITY_NONE)

In [14]:
ser.write(('/home\r\n').encode('utf-8'))
qry = ser.readline()
print(qry)


b'@01 0 OK BUSY FE 0\r\n'


In [15]:
# periodic movement with triggers

# from example
#     /1 trigger 1 when 1 pos == 0↵ 
#     /1 trigger 1 action a 1 move abs 100000↵ 
#     /1 trigger 1 action b 1 maxspeed = 150000↵ 
#     /1 trigger 2 when 1 pos == 100000↵ 
#     /1 trigger 2 action a 1 move abs 0↵ 
#     /1 trigger 2 action b 1 maxspeed = 75000↵ 
#     /1 trigger 1 enable↵ 
#     /1 trigger 2 enable↵ 

# def periodic_different_speed(self, scan_range = 1e-3, vel1 = 20, vel2 = 40)
# self.scan_range = scan_range
# self.vel1 = vel1
# self.vel2 = vel2
pos1 = 5.5 # mm
pos2 = 6.5 # mm
vel1 = 20 # mm/s
vel2 = 40 # mm/s
accel = 800 # mm/s^2

stepsize = 2e-4
vel_factor = 8192
accel_facror = 1.220703125

# set current
ser.write(('/set driver.current.run 7\r\n').encode('utf-8'))
qry = ser.readline()
print(qry)

# set acceleration
ser.write(('/set accel {:.0f}\r\n'.format(accel * accel_facror)).encode('utf-8'))
qry = ser.readline()
print(qry)



# set periodic movement using triggers
ser.write(('/trigger 1 when 1 pos == {:.0f}\r\n'.format(pos1/stepsize)).encode('utf-8'))
qry = ser.readline()
ser.write(('/trigger 2 when 1 pos == {:.0f}\r\n'.format(pos2/stepsize)).encode('utf-8'))
qry = ser.readline()
ser.write(('/trigger 1 action a 1 move abs {:.0f}\r\n'.format(pos2/stepsize)).encode('utf-8'))
qry = ser.readline()
ser.write(('/trigger 2 action a 1 move abs {:.0f}\r\n'.format(pos1/stepsize)).encode('utf-8'))
qry = ser.readline()
ser.write(('/trigger 1 action b 1 maxspeed = {:.0f}\r\n'.format(vel1*vel_factor)).encode('utf-8'))
qry = ser.readline()
ser.write(('/trigger 2 action b 1 maxspeed = {:.0f}\r\n'.format(vel2*vel_factor)).encode('utf-8'))
qry = ser.readline()
#enable triggers
ser.write(('/trigger 1 enable\r\n').encode('utf-8'))
qry = ser.readline()
ser.write(('/trigger 2 enable\r\n').encode('utf-8'))
qry = ser.readline()

# start movement by moving to the trigger position
ser.write(('/move abs {:.0f}\r\n'.format(pos1/stepsize)).encode('utf-8'))
qry = ser.readline()





b'@01 0 OK IDLE FE 0\r\n'
b'@01 0 OK IDLE FE 0\r\n'


In [16]:
# stop movement -> disable triggers
ser.write(('/trigger 1 disable\r\n').encode('utf-8'))
qry = ser.readline()
ser.write(('/trigger 2 disable\r\n').encode('utf-8'))
qry = ser.readline()



In [123]:
ser.write(('/move vel 1000 \r\n').encode('utf-8'))

17

In [45]:
ser.write(('/stop \r\n').encode('utf-8'))

8

In [25]:
ser.write(('/move abs 25000 \r\n').encode('utf-8'))
qry = ser.readline()
print(qry)    

# wait till iddle
wait = True;
while wait:
    ser.write(('/get deviceid\r\n').encode('utf-8'))
    qry = ser.readline()        
    if qry.find(b'IDLE') != -1:
        wait = False
    time.sleep(0.1)



ser.write(('/move abs 10000 \r\n').encode('utf-8'))
qry = ser.readline()
print(qry)

b'@01 0 OK BUSY -- 0\r\n'
b'@01 0 OK BUSY -- 0\r\n'


In [26]:
ser.write(('/move sin 10000 1000\r\n').encode('utf-8'))
qry = ser.readline()
print(qry)

b'@01 0 OK BUSY -- 0\r\n'


In [17]:
ser.close()

In [198]:
'/move abs {:.2f} {} \r\n'.format(25.2545,23.3587)

'/move abs 25.255  \r\n'